In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... - done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=3986071272c465c871d253f0f0ff2675027b9995832462aa72e5a0abbf8eb9a7
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [2]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import math
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import re
import scipy
import math
import wikipedia
import requests
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.fasttext import FastText

2024-07-18 16:58:13.231304: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 16:58:13.231403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 16:58:13.397799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **GET DOCS FROM WIKIPEDIA**

In [3]:
titles=wikipedia.search('spiderman')
result=wikipedia.page(titles)

In [4]:
print("len of docs",len(result.content))
print("type of docs",type(result.content))
print("content",result.content)

len of docs 860
type of docs <class 'str'>
content Spiderman of the Rings is the first studio album by the American electronic musician Dan Deacon, released by Carpark Records on May 8, 2007. The album was released on white vinyl by Wildfire Wildfire Records.


== Track listing ==
"Wooody Wooodpecker" — 3:50
"The Crystal Cat" — 3:53
"Wham City" — 11:45
"Big Milk" — 4:25
"Okie Dokie" — 2:37
"Trippy Green Skull" — 4:00
"Snake Mistakes" — 4:11
"Pink Batman" — 5:04
"Jimmy Joe Roche" — 5:58


== Reception ==

The single from this album, "The Crystal Cat," was #84 on Rolling Stone's list of the 100 Best Songs of 2007, and "Wham City" was #30 on Pitchfork's "Top 100 Tracks of 2007."


== Personnel ==
Musicians
Dan Deacon – Primary Artist
Adam Endres – Vocals
Dina Kelberman – Vocals
Connor Kizer – Vocals
Ed Schrader – Vocals
Production
OCDJ – Producer
Kevin O’Meara – Production Assistant


== References ==


In [5]:
def charbased_preprocessing(content):
    tokens = []
    for char in content:
        tokens.append(char)
    return(tokens)
    

In [6]:
def wordbased_prepocessing(text, stopWords = set(stopwords.words('english'))):
    text=text.lower()
    text=text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\w*\d\w*' , ' ' , text)
    text = re.sub(r'[0-9]' , ' ' , text)
    text = re.sub(r'[^a-zA-Z0-9\s]' , ' ' , text)
    text = re.sub(r'\s+' , ' ', text)
    text = word_tokenize(text)
    text = [i for i in text if i != '']
    return text

In [7]:
chars=np.array(charbased_preprocessing(result.content)).reshape(-1,1)

In [8]:
encoder = OneHotEncoder()
chars = encoder.fit_transform(chars).toarray()
print(chars)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
chars.shape

(860, 59)

In [10]:
chars[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0

In [11]:
x=[]
y=[]
sequence_length = 10
for i in range(0,chars.shape[0]-sequence_length):
    x.append(chars[i:i+sequence_length])
    y.append(chars[i+sequence_length])
x=np.array(x)
y=np.array(y)

In [12]:
print(x.shape)
print(y.shape)

(850, 10, 59)
(850, 59)


In [13]:
dic = set([c for c in result.content])
dic=sorted(dic)
index2char = dict((i, c) for i, c in enumerate(dic))

In [14]:
index2char

{0: '\n',
 1: ' ',
 2: '"',
 3: '#',
 4: "'",
 5: ',',
 6: '.',
 7: '0',
 8: '1',
 9: '2',
 10: '3',
 11: '4',
 12: '5',
 13: '7',
 14: '8',
 15: ':',
 16: '=',
 17: 'A',
 18: 'B',
 19: 'C',
 20: 'D',
 21: 'E',
 22: 'G',
 23: 'J',
 24: 'K',
 25: 'M',
 26: 'O',
 27: 'P',
 28: 'R',
 29: 'S',
 30: 'T',
 31: 'V',
 32: 'W',
 33: 'a',
 34: 'b',
 35: 'c',
 36: 'd',
 37: 'e',
 38: 'f',
 39: 'g',
 40: 'h',
 41: 'i',
 42: 'k',
 43: 'l',
 44: 'm',
 45: 'n',
 46: 'o',
 47: 'p',
 48: 'r',
 49: 's',
 50: 't',
 51: 'u',
 52: 'v',
 53: 'w',
 54: 'y',
 55: 'z',
 56: '–',
 57: '—',
 58: '’'}

In [15]:
import tensorflow as tf

# Define the size of the input and output layers
input_size = chars.shape[1] 
output_size = chars.shape[1]
# Define the hyperparameters
hidden_size = 100  # Size of the hidden state
sequence_length = 10  # Length of the input sequence
learning_rate = 0.001

# Define the RNN model with multiple layers
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True, input_shape=(sequence_length, input_size)),
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True),
    tf.keras.layers.SimpleRNN(hidden_size),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 10, 100)        │        16,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 10, 100)        │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 59)             │         5,959 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,159 (242.81 KB)

 Trainable params: 62,159 (242.81 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history=model.fit(x, y, batch_size=64, epochs=50)

Epoch 1/50
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0279 - loss: 4.1370 

I0000 00:00:1721321928.938260      83 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - accuracy: 0.0314 - loss: 4.1238
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1375 - loss: 3.5001
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2256 - loss: 3.1065
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2980 - loss: 2.8578
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3370 - loss: 2.6320
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4377 - loss: 2.3610
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4945 - loss: 2.1949
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5366 - loss: 1.9995
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5636 - loss: 1.8528
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6422 - loss: 1.7062
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6676 - loss: 1.5367
Epoch 12/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7844 - loss: 1.322

# **Predict Next Char**

In [17]:
# Choose a random sequence from x
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

# Reshape the random sequence to fit the model input shape
random_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))

predicted_char_prob = model.predict(random_sequence)
predicted_char_index = np.argmax(predicted_char_prob)

# Convert the predicted character index to the actual character using index2char dictionary
predicted_char = index2char[predicted_char_index]

# Convert the random sequence back to characters using index2char dictionary
sequence_chars = [index2char[np.argmax(char)] for char in random_sequence[0]]

# Print the random sequence and the predicted character
print("Random Sequence:", ''.join(sequence_chars))
print("Predicted Next Character:", predicted_char)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step
Random Sequence: tone's lis
Predicted Next Character: t


# **Generate a Text**

In [18]:
sequence_index = np.random.randint(0, len(chars) - sequence_length)
sequence_chars = [index2char[np.argmax(char)] for char in chars[sequence_index:sequence_index + sequence_length]]

generated_text = ''.join(sequence_chars) 
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

for _ in range(100):
    current_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))
    predicted_char_prob = model.predict(current_sequence)
    predicted_char_index = np.argmax(predicted_char_prob)
    predicted_char = index2char[predicted_char_index]
    generated_text += predicted_char
    encoded_char=np.zeros(random_sequence.shape[1])
    encoded_char[predicted_char_index]=1
    random_sequence =np.vstack((random_sequence[1:],encoded_char))

print("Generated Text:", generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

In [19]:
history=model.fit(x, y, batch_size=64, epochs=150)


Epoch 1/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9895 - loss: 0.0702
Epoch 2/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9884 - loss: 0.0689
Epoch 3/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9849 - loss: 0.0614
Epoch 4/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9805 - loss: 0.0675
Epoch 5/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9910 - loss: 0.0523
Epoch 6/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9857 - loss: 0.0536
Epoch 7/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9856 - loss: 0.0550
Epoch 8/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9840 - loss: 0.0594
Epoch 9/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9887 - loss: 0.0489
Epoch 10/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9882 - loss: 0.0544
Epoch 11/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9854 - loss: 0.0561
Epoch 12/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy:

In [20]:
random_sequence = x[random_index]

for _ in range(100):
    current_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))
    predicted_char_prob = model.predict(current_sequence)
    predicted_char_index = np.argmax(predicted_char_prob)
    predicted_char = index2char[predicted_char_index]
    generated_text += predicted_char
    encoded_char=np.zeros(random_sequence.shape[1])
    encoded_char[predicted_char_index]=1
    random_sequence =np.vstack((random_sequence[1:],encoded_char))

print("Generated Text:", generated_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

# **word based**

In [21]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
words= wordbased_prepocessing(result.content)
print(len(words))
print(words)

111
['spiderman', 'of', 'the', 'rings', 'is', 'the', 'first', 'studio', 'album', 'by', 'the', 'american', 'electronic', 'musician', 'dan', 'deacon', 'released', 'by', 'carpark', 'records', 'on', 'may', 'the', 'album', 'was', 'released', 'on', 'white', 'vinyl', 'by', 'wildfire', 'wildfire', 'records', 'track', 'listing', 'wooody', 'wooodpecker', 'the', 'crystal', 'cat', 'wham', 'city', 'big', 'milk', 'okie', 'dokie', 'trippy', 'green', 'skull', 'snake', 'mistakes', 'pink', 'batman', 'jimmy', 'joe', 'roche', 'reception', 'the', 'single', 'from', 'this', 'album', 'the', 'crystal', 'cat', 'was', 'on', 'rolling', 'stones', 'list', 'of', 'the', 'best', 'songs', 'of', 'and', 'wham', 'city', 'was', 'on', 'pitchforks', 'top', 'tracks', 'of', 'personnel', 'musicians', 'dan', 'deacon', 'primary', 'artist', 'adam', 'endres', 'vocals', 'dina', 'kelberman', 'vocals', 'connor', 'kizer', 'vocals', 'ed', 'schrader', 'vocals', 'production', 'ocdj', 'producer', 'kevin', 'o', 'meara', 'production', 'assis

In [23]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
! gunzip "cc.en.300.bin.gz"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-07-18 16:59:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.239.50.18, 18.239.50.104, 18.239.50.9, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.239.50.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: 'cc.en.300.bin.gz'

cc.en.300.bin.gz    100%[===================>]   4.19G   268MB/s    in 15s     

2024-07-18 16:59:38 (288 MB/s) - 'cc.en.300.bin.gz' saved [4503593528/4503593528]



In [24]:
from gensim.models.fasttext import load_facebook_model

In [25]:
pre_model=load_facebook_model('/kaggle/working/cc.en.300.bin')

In [26]:
word_vector=[]
for i in words:
    word_vector.append(pre_model.wv[i])
word_vector=np.array(word_vector)
word_vector

array([[ 1.3786110e-03,  1.7122723e-02, -3.3165865e-02, ...,
         5.3082127e-02, -6.0556062e-02, -1.0652682e-01],
       [-7.3038238e-05, -1.8774074e-01, -7.1051158e-02, ...,
         2.8464055e-02, -1.3843335e-01,  1.8053975e-02],
       [-5.1744193e-02,  7.3963955e-02, -1.3056880e-02, ...,
         2.3702517e-01,  4.4751912e-04, -4.1930666e-03],
       ...,
       [ 2.8702345e-02,  3.9084632e-02,  4.3092314e-03, ...,
         4.7602560e-02, -1.1586945e-02, -2.1473607e-03],
       [ 6.3735239e-02,  2.6593592e-02,  2.2695802e-02, ...,
         5.7959970e-02,  1.8718690e-02,  1.4781803e-03],
       [ 2.2990290e-02,  3.9157346e-02,  2.0146200e-02, ...,
         1.4130569e-02, -2.2140609e-02, -1.0044541e-02]], dtype=float32)

In [27]:
word_vector.shape

(111, 300)

In [28]:
def find_closest_word(embedding, words, pre_model):
    closest_word = None
    min_distance = float('inf')
    for word in words:
        emb = pre_model.wv[word]
        distance = np.linalg.norm(emb - embedding)
        if distance < min_distance:
            closest_word = word
            min_distance = distance
    return closest_word

In [29]:
x=[]
y=[]
sequence_length = 10
for i in range(0,word_vector.shape[0]-sequence_length):
    x.append(word_vector[i:i+sequence_length])
    y.append(word_vector[i+sequence_length])
x=np.array(x)
y=np.array(y)

In [30]:
print(x.shape)
print(y.shape)

(101, 10, 300)
(101, 300)


In [31]:
import tensorflow as tf

# Define the size of the input and output layers
input_size = word_vector.shape[1] 
output_size = word_vector.shape[1]
# Define the hyperparameters
hidden_size = 100  # Size of the hidden state
sequence_length = 10  # Length of the input sequence
learning_rate = 0.001

# Define the RNN model with multiple layers
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True, input_shape=(sequence_length, input_size)),
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True),
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True),
    tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True),
    tf.keras.layers.SimpleRNN(hidden_size),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='mse',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_3 (SimpleRNN)        │ (None, 10, 100)        │        40,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ (None, 10, 100)        │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 10, 100)        │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_6 (SimpleRNN)        │ (None, 10, 100)        │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_7 (SimpleRNN)        │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 300)            │        30,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 150,800 (589.06 KB)

 Trainable params: 150,800 (589.06 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
history=model.fit(x, y, batch_size=32, epochs=50)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 924ms/step - accuracy: 0.0060 - loss: 0.0091
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0369 - loss: 0.0098 
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0661 - loss: 0.0091
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2019 - loss: 0.0094
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2586 - loss: 0.0096
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2846 - loss: 0.0090
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2398 - loss: 0.0087
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2190 - loss: 0.0086
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2398 - loss: 0.0091
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2242 - loss: 0.0090
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2713 - loss: 0.0090
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2734 - loss: 0.0086

In [33]:
# Choose a random sequence from x
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

# Reshape the random sequence to fit the model input shape
random_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))

predicted_word_emb = model.predict(random_sequence)

# Convert the predicted character index to the actual character using index2char dictionary
predicted_word = find_closest_word(predicted_word_emb, words, pre_model)
#sequence_words = [pre_model.get_nearest_neighbors(vector, k=1)[0][0] for vector in random_sequence[0]]
sequence_words = [find_closest_word(vector, words, pre_model) for vector in random_sequence[0]]


# Print the random sequence and the predicted character
print("Random Sequence:", ' '.join(sequence_words))
print("Predicted Next word:", predicted_word)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Random Sequence: this album the crystal cat was on rolling stones list
Predicted Next word: and


In [34]:
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]
sequence_words = [find_closest_word(vector, words, pre_model) for vector in random_sequence]
generated_text=' '.join(sequence_words)
print(generated_text)
for _ in range(10):
    current_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))
    predicted_word_emb = model.predict(current_sequence)
    predicted_word=find_closest_word(predicted_word_emb, words, pre_model)
    generated_text +=' '+predicted_word
    random_sequence =np.vstack((random_sequence[1:],predicted_word_emb))

print("Generated Text:", generated_text)


from this album the crystal cat was on rolling stones
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Generated Text: from this album the crystal cat was on rolling stones kelberman and kelberman kelberman kelberman and kelberman kelberman kelberman kelberman


In [35]:
history=model.fit(x, y, batch_size=64, epochs=150)


Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.3536 - loss: 0.0077
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3640 - loss: 0.0080
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3730 - loss: 0.0080
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3404 - loss: 0.0083
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3730 - loss: 0.0078
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3915 - loss: 0.0087
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3838 - loss: 0.0081
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4165 - loss: 0.0080
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4165 - loss: 0.0081
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4113 - loss: 0.0083
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4022 - loss: 0.0084
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4154 - loss

In [36]:
# Choose a random sequence from x
random_index = np.random.randint(0, len(x))
random_sequence = x[random_index]

# Reshape the random sequence to fit the model input shape
random_sequence = np.reshape(random_sequence, (1, sequence_length, input_size))

predicted_word_emb = model.predict(random_sequence)

# Convert the predicted character index to the actual character using index2char dictionary
predicted_word = find_closest_word(predicted_word_emb, words, pre_model)
#sequence_words = [pre_model.get_nearest_neighbors(vector, k=1)[0][0] for vector in random_sequence[0]]
sequence_words = [find_closest_word(vector, words, pre_model) for vector in random_sequence[0]]


# Print the random sequence and the predicted character
print("Random Sequence:", ' '.join(sequence_words))
print("Predicted Next word:", predicted_word)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Random Sequence: the best songs of and wham city was on pitchforks
Predicted Next word: kelberman
